# 第7章: 単語ベクトル
単語の意味を実ベクトルで表現する単語ベクトル（単語埋め込み）に関して，以下の処理を行うプログラムを作成せよ．

## 60. 単語ベクトルの読み込みと表示
Google Newsデータセット（約1,000億単語）での学習済み単語ベクトル（300万単語・フレーズ，300次元）をダウンロードし，”United States”の単語ベクトルを表示せよ．<br>ただし，”United States”は内部的には”United_States”と表現されていることに注意せよ．

In [2]:
from gensim.models import KeyedVectors

# 学習済みのWord2Vecモデルを読み込む
model = KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin.gz', binary=True)

model['United_States']


array([-3.61328125e-02, -4.83398438e-02,  2.35351562e-01,  1.74804688e-01,
       -1.46484375e-01, -7.42187500e-02, -1.01562500e-01, -7.71484375e-02,
        1.09375000e-01, -5.71289062e-02, -1.48437500e-01, -6.00585938e-02,
        1.74804688e-01, -7.71484375e-02,  2.58789062e-02, -7.66601562e-02,
       -3.80859375e-02,  1.35742188e-01,  3.75976562e-02, -4.19921875e-02,
       -3.56445312e-02,  5.34667969e-02,  3.68118286e-04, -1.66992188e-01,
       -1.17187500e-01,  1.41601562e-01, -1.69921875e-01, -6.49414062e-02,
       -1.66992188e-01,  1.00585938e-01,  1.15722656e-01, -2.18750000e-01,
       -9.86328125e-02, -2.56347656e-02,  1.23046875e-01, -3.54003906e-02,
       -1.58203125e-01, -1.60156250e-01,  2.94189453e-02,  8.15429688e-02,
        6.88476562e-02,  1.87500000e-01,  6.49414062e-02,  1.15234375e-01,
       -2.27050781e-02,  3.32031250e-01, -3.27148438e-02,  1.77734375e-01,
       -2.08007812e-01,  4.54101562e-02, -1.23901367e-02,  1.19628906e-01,
        7.44628906e-03, -

## 61. 単語の類似度
“United States”と”U.S.”のコサイン類似度を計算せよ．

In [3]:
model.similarity('United_States', 'U.S.')

0.73107743

## 62. 類似度の高い単語10件
“United States”とコサイン類似度が高い10語と，その類似度を出力せよ.

In [4]:
model.most_similar('United_States', topn=10)

[('Unites_States', 0.7877248525619507),
 ('Untied_States', 0.7541370987892151),
 ('United_Sates', 0.7400724291801453),
 ('U.S.', 0.7310774326324463),
 ('theUnited_States', 0.6404393911361694),
 ('America', 0.6178410053253174),
 ('UnitedStates', 0.6167312264442444),
 ('Europe', 0.6132988929748535),
 ('countries', 0.6044804453849792),
 ('Canada', 0.601906955242157)]

## 63. 加法構成性によるアナロジー
“Spain”の単語ベクトルから”Madrid”のベクトルを引き，”Athens”のベクトルを足したベクトルを計算し，<br>そのベクトルと類似度の高い10語とその類似度を出力せよ.

In [8]:
# ベクトルの計算によって
vector = model['Spain'] - model['Madird'] + model['Athens']
model.most_similar(vector, topn=10)

[('Athens', 0.8415753841400146),
 ('Spain', 0.7456149458885193),
 ('Greece', 0.6866837739944458),
 ('Athens_Greece', 0.6324047446250916),
 ('Madrid', 0.5868974328041077),
 ('Thessaloniki', 0.5800884366035461),
 ('Mykonos_Island', 0.579736053943634),
 ('Portugal', 0.5660486817359924),
 ('Heraklio', 0.5607272386550903),
 ('Greeks', 0.5496171712875366)]

In [9]:
# 単語の意味によって

model.most_similar(positive=['Spain', 'Athens'], negative=['Madird'], topn=10)

[('Greece', 0.6015121340751648),
 ('Athens_Greece', 0.5763437151908875),
 ('Thessaloniki', 0.508480429649353),
 ('Greeks', 0.4871496558189392),
 ('Greek', 0.47021713852882385),
 ('Lithuania', 0.466966450214386),
 ('Mykonos_Island', 0.4635937809944153),
 ('Iraklion', 0.4567350149154663),
 ('Organising_Committee_ATHOC', 0.4564575254917145),
 ('Rome', 0.45040011405944824)]

## 64. アナロジーデータでの実験
単語アナロジーの評価データをダウンロードし，vec(2列目の単語) - vec(1列目の単語) + vec(3列目の単語)を計算し，<br>そのベクトルと類似度が最も高い単語と，その類似度を求めよ．<br>求めた単語と類似度は，各事例の末尾に追記せよ．

In [11]:
with open('questions-words.txt', 'r') as f1, open('questions-words-add.txt', 'w') as f2:
    for line in f1:
        line = line.split()
        if line[0] == ':':
            category = line[1]
        else:
            word, val = model.most_similar(positive=[model[line[1]], model[line[2]]], negative=[model[line[0]]], topn=1)[0]
            f2.write(' '.join([category] + line + [word, str(val) + '\n']))